In [1]:
import pandas as pd
import geopandas as gpd
import folium
from pysal.explore import esda
from pysal.lib import weights
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import contextily as ctx

# Load the Louisiana parish shape file
parishes_geo = gpd.read_file('C:\\Users\\scfra\\OneDrive\\Desktop\\Final Datasets\\Louisiana_Parishes.shp')

# Select the parishes that the model will cover
parishes = ['Ascension', 'Caddo', 'Calcasieu', 'East Baton Rouge', 'Iberville',
            'Jefferson', 'Lafayette', 'Ouachita', 'Rapides', 'St. Bernard',
            'Tangipahoa', 'Terrebonne', 'West Baton Rouge']

# Filter the parishes_geo GeoDataFrame to include only the parishes present in the datasets
parishes_geo_filtered = parishes_geo[parishes_geo['Name'].isin(parishes)]

def create_maps(year):
    # Load the PM2.5 data for the selected year
    pm25_file = f'C:\\Users\\scfra\\OneDrive\\Desktop\\Final Datasets\\pm2.5_{year}.csv'
    data = pd.read_csv(pm25_file)
    
    # Load the lung cancer data for the selected year
    cancer_file = f'C:\\Users\\scfra\\OneDrive\\Desktop\\Final Datasets\\lung_cancer_{year}.csv'
    cancer_data = pd.read_csv(cancer_file)
    
    # Filter the data for the selected year and specified parishes
    data_filtered = data[(data['Year'] == year) & (data['Name'].isin(parishes))]
    cancer_data_filtered = cancer_data[(cancer_data['Year'] == year) & (cancer_data['Name'].isin(parishes))]
    
    # Merge the PM2.5 data with the filtered parish boundary data
    merged_data = parishes_geo_filtered.merge(data_filtered, left_on='Name', right_on='Name')
    
    # Merge the lung cancer data
    merged_data = merged_data.merge(cancer_data_filtered, left_on='Name', right_on='Name')
    
    # Calculate Moran's I for spatial autocorrelation
    w = weights.Queen.from_dataframe(merged_data)
    moran_pm25 = esda.Moran(merged_data['Mean'], w)
    moran_cancer = esda.Moran(merged_data['Rate'], w)
    
    # Create the non-interactive map for PM2.5 mean data 
    fig, ax = plt.subplots(figsize=(10, 8))
    merged_data.plot(ax=ax, edgecolor='black', column='Rate', cmap='Reds', legend=True)
    merged_data.centroid.plot(ax=ax, markersize=merged_data['Mean'], color='black', alpha=0.7)
    ctx.add_basemap(ax)
    ax.set_title(f'PM2.5 Mean for Louisiana Parishes in {year}')
    ax.set_axis_off()
    
    # Create the interactive map centered on Louisiana for Lung Cancer
    louisiana_coords = [30.9843, -91.9623] # Coordinates for the center of Louisiana
    interactive_map = folium.Map(location=louisiana_coords, zoom_start=7)
    
    # Add the filtered parish polygons to the map
    folium.GeoJson(
        parishes_geo_filtered,
        name='Parishes',
        style_function=lambda feature: {
            'fillColor': 'white',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.2
        }
    ).add_to(interactive_map)
    
    # Add parish polygons to the map and shade them based on lung cancer rates
    folium.Choropleth(
        geo_data=merged_data,
        name='Lung Cancer Rates',
        data=merged_data,
        columns=['Name', 'Rate'],
        key_on='feature.properties.Name',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Lung Cancer Rates'
    ).add_to(interactive_map)
    
    # Create a feature group for PM2.5 mean values
    pm25_group = folium.FeatureGroup(name='PM2.5 Mean Values')
    
    # Add circles for PM2.5 mean values
    for idx, row in merged_data.iterrows():
        folium.Circle(
            location=[row.geometry.centroid.y, row.geometry.centroid.x],
            radius=5000, # Adjust the radius as needed
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=1,
            tooltip=f"Parish: {row['Name']}<br>PM2.5 Mean: {row['Mean']:.2f}"
        ).add_to(pm25_group)
    
    # Add the PM2.5 feature group to the map
    pm25_group.add_to(interactive_map)
    
    # Add a layer control to the map
    folium.LayerControl().add_to(interactive_map)
    
    # Display Moran's I values
    print(f"Moran's I for PM2.5 in {year}: {moran_pm25.I:.4f}")
    print(f"Moran's I p-value for PM2.5 in {year}: {moran_pm25.p_sim:.4f}")
    print(f"Moran's I for Lung Cancer Rates in {year}: {moran_cancer.I:.4f}")
    print(f"Moran's I p-value for Lung Cancer Rates in {year}: {moran_cancer.p_sim:.4f}")
    
    return fig, interactive_map

# Create the interactive dashboard
@interact(year=IntSlider(min=2011, max=2020, step=1, value=2011))
def display_dashboard(year):
    fig, interactive_map = create_maps(year)
    plt.show(fig)
    display(interactive_map)

C:\Users\scfra\anaconda3\Lib\site-packages\spaghetti\network.py:42: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(dep_msg, FutureWarning, stacklevel=1)


interactive(children=(IntSlider(value=2011, description='year', max=2020, min=2011), Output()), _dom_classes=(…